In [1]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from models import *
import time
import re, io, nltk, torch 
from nltk.corpus import stopwords
from numpy import linalg as LA
from numpy.linalg import norm
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
from sklearn.utils.class_weight import compute_class_weight
from gensim import utils, matutils  
from gensim.corpora.dictionary import Dictionary
from six import string_types, integer_types
from six.moves import zip, range
from numpy import linalg as LA
from sklearn.preprocessing import Normalizer
from scipy import stats
from gensim.utils import deprecated
from numpy import dot, float32 as REAL, memmap as np_memmap, \
    double, array, zeros, vstack, sqrt, newaxis, integer, \
    ndarray, sum as np_sum, prod, argmax
from collections import Counter
from scipy.spatial import distance
from numpy.random import binomial
from scipy.stats import bernoulli
import numpy as np


n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('There are',torch.cuda.device_count(),'GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt


dict_cl=dict()
dict_cl[0]='support'
dict_cl[1]='denying'
dict_cl[2]='questioning'
dict_cl[3]='commenting'

#sdqc
d_lab=dict()
d_lab["support"]=0
d_lab["denying"]=1
d_lab["questioning"]=2
d_lab["commenting"]=3

train =pd.read_csv("../Tesis_exp/Fine-Tuning/CSV_Stance/train_semeval_raw.csv")
val=pd.read_csv("../Tesis_exp/Fine-Tuning/CSV_Stance/dev_semeval_raw.csv")
test=pd.read_csv("../Tesis_exp/Fine-Tuning/CSV_Stance/test_semeval_raw.csv")


Using TensorFlow backend.
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [2]:
import re
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

stemmer_sn = SnowballStemmer("english")
stemmer = PorterStemmer()

stoplist = stopwords.words("english")
lemmatizer=WordNetLemmatizer()

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

def cleaner(post): 
    #re.sub(r'([a-z])\1+', r'\1', "user i think that ' s all you loooooove ")
    s= re.sub(r"http\S+", " html ", post)   ##########elimina http    
    s= re.sub(r'#\w+ ?', " hashtag ", s) ########3
    s= re.sub(r'@\w+ ?', " user ", s)##############    
    s= s.lower()
    s=emoji_pattern.sub(r'', s)
    s=re.sub(r"'\b", " ' ", s)
    s=re.sub(r"\b'\b", " ' ", s)
    s=re.sub(r"“\b", " “ ", s)
    #“
    s=re.sub(r"\b’", " ’ ", s)
    s=re.sub(r"‘\b", " ‘ ", s)
    s=re.sub(r"\b’\b", " ‘ ", s)
    s = re.sub(r"-", " - ", s)
    s = re.sub(r"\(", " ", s)
    s = re.sub(r"\)", " ", s)
    s = re.sub(r"\?", " ? ", s)    
    s = re.sub(r"\/", " ", s)
    s = re.sub(r"' ", " ' ", s)
    s = re.sub(r" '", " ' ", s)
    s = re.sub(r"\!", " ! ", s)
    s=re.sub("[\.]+", " . ", s)
    s=re.sub("[\,]+", " , ", s)
    s=re.sub("[\;]+", " ; ", s)
    s=re.sub("[\:]+", " : ", s)
    s=re.sub('[\"]+', ' " ', s)
    s=re.sub(r'\b[0-9]\b', " number ",  s)
    s=re.sub(r'\b[0-9]*[0-9]\b', " number ",  s)    
    s=re.sub(r'\b”', ' " ', s)
    sl= list(s.split())
    sl_2=[]
    for wd in sl:
        try: 
            q=Word2Index[wd]
            sl_2.append(wd)
        except:
            try: 
                if stemmer.stem(wd) in Word2Index.keys():
                    sl_2.append(stemmer.stem(wd))
                elif lemmatizer.lemmatize(wd) in Word2Index.keys():
                    sl_2.append(lemmatizer.lemmatize(wd))
                else:
                    sl_2.append(wd)
            except:
                sl_2.append(wd)
    sl=sl_2
    s=' '.join([word for word in sl])# if word not in stoplist])
    return s, sl

embeddings_index = {}
f = open(os.path.join("glove.twitter.27B.200d.txt"))#'glove.twitter.27B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Se encontraron %s terminos con sus vectores de embedding.' % len(embeddings_index))

Se encontraron 1193514 terminos con sus vectores de embedding.


In [3]:
word_index=dict()
j=1
for frase in train["Tweet"].values:
    frase=cleaner(frase)[-1]
    #print (frase)
    for term in frase:
        if term not in word_index.keys():
            word_index[term]=j
            j+=1

for frase in val["Tweet"].values:
    frase=cleaner(frase)[-1]
    #print (frase)
    for term in frase:
        if term not in word_index.keys():
            word_index[term]=j
            j+=1
            
for frase in test["Tweet"].values:
    frase=cleaner(frase)[-1]
    #print (frase)
    for term in frase:
        if term not in word_index.keys():
            word_index[term]=j
            j+=1
            
len(word_index.keys())

7236

In [4]:
not_found=[]
pos_tag_nf=[]
Word2Index={}
idx_2_token={}

g_dim=200
emb_matrix = np.zeros((len(word_index.keys())+1, g_dim))  

for word, i in word_index.items():
    vector = embeddings_index.get(word)
    if vector is not None:
        # words sin match en Glove, serán vectores de ceros.
        emb_matrix[i] = vector
        Word2Index[word]=i
        idx_2_token[i]=word
    else:
        Word2Index[word]=i
        idx_2_token[i]=word
        not_found.append(word)
        pos_tag_nf.append(nltk.pos_tag([word])[0][-1])

In [5]:
transf = Normalizer().fit(emb_matrix) 
M_GLOVE_space=transf.transform(emb_matrix)

def my_tokenizer(lista):
    to_return=[]
    for l in lista:
        if l in word_index.keys():
            to_return.append(l)
        else:
            continue
    return to_return

def predict_data(trained, x_train, x_val, x_test, etiq, etiq_v, etiq_t, name_model):
    etiq = etiq.astype("int")
    trainPredict = trained.predict(x_train, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq, trainPredict)  
    f1=f1_score(etiq, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    #matriz=normalize(confusion_matrix(etiq, trainPredict))
    print ("")
    print ("Accuracy sobre Train", name_model, ":",acc)  
    print ("F1-score None sobre Train", name_model, ":",f1)
    print ("F1-score macro sobre Train", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_val, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq_v, trainPredict)  
    f1=f1_score(etiq_v, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq_v, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    #matriz=normalize(confusion_matrix(etiq_v, trainPredict))
    print ("")
    print ("Accuracy sobre Val", name_model, ":",acc)  
    print ("F1-score None sobre Val", name_model, ":",f1)
    print ("F1-score macro sobre Val", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_test, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc_t= accuracy_score(etiq_t, trainPredict)  
    f1_t=f1_score(etiq_t, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma_t=f1_score(etiq_t, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    matriz_t=normalize(confusion_matrix(etiq_t, trainPredict))
    print ("")
    print ("Accuracy sobre Test", name_model, ":",acc_t)  
    print ("F1-score None sobre Test", name_model, ":",f1_t)
    print ("F1-score macro sobre Test", name_model, ":",f1_ma_t)
    
    return f1_ma_t, f1_t, acc_t, matriz_t

In [6]:
from models import *

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
class Dummy_Embeddings(nn.Module):
    def __init__(self, d_model, dummy_vectors): 
        super(Dummy_Embeddings, self).__init__()
        aux = torch.from_numpy(dummy_vectors)#, dtype='float32'))
        self.index2dummy = nn.Embedding(aux.size()[0], d_model)
        self.index2dummy.weigth=nn.Parameter(aux)
        self.index2dummy.weigth.requires_grad=False
        self.d_model = d_model
        
    def forward(self, x):
        aux=x.numpy()
        #print ("aux original", aux)
        #print ("aux restado...",aux-np.ones(aux.shape))
        new_x= aux #- np.ones(aux.shape)
        new_x= torch.from_numpy(new_x)
        return self.index2dummy(new_x.long()) * math.sqrt(self.d_model) #debiese retornar matriz de batch_size x [ind_tw, k1,k2,k3,k4,k5,k6] (si son 6 modelos)
    
def match(objetos,ejemplo):
    i=0
    for obj in objetos:
        if obj==ejemplo:
            return i
        i+=1
        
def plot_confusion_matrix(cm, target_names, title='Confusion matrix (f1-score)',cmap=None, normalize=True):    
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if cmap is None:
        cmap = plt.get_cmap('Blues')
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)
    plt.title(title)
    plt.colorbar()
    
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    
MAX_LEN = 37

print ("Conteo por stance val\n", Counter(val['Label']))
print ("Conteo por stance test\n", Counter(test['Label']))


Conteo por stance val
 Counter({'commenting': 173, 'support': 69, 'questioning': 28, 'denying': 11})
Conteo por stance test
 Counter({'commenting': 778, 'questioning': 106, 'denying': 69, 'support': 68})


In [7]:
ids_val=val['Tw_id']
tws_val=val['Tweet']
labels_val=[d_lab[lb] for lb in val['Label']]

etiq_v=labels_val.copy()
etiq_v=np.asarray(etiq_v)
sentences_val = tws_val
n_labels_val = np.array(etiq_v)
y_val=to_categorical(n_labels_val,num_classes=4)

input_ids_val=[]
for sent in sentences_val:
    sent_cl=cleaner(sent)[1]
    encoded_sent=[]
    for wd in sent_cl:
        try: 
            encoded_sent.append(Word2Index[wd])
        except: 
            continue   
    input_ids_val.append(encoded_sent)
    
ids_test=test['Tw_id']
tws_test=test['Tweet']
labels_test=[d_lab[lb] for lb in test['Label']]

etiq_t=labels_test.copy()
etiq_t=np.asarray(etiq_t)
sentences_test = tws_test
n_labels_test = np.array(etiq_t)
y_test=to_categorical(n_labels_test,num_classes=4)

input_ids_test=[]
for sent in sentences_test:
    sent_cl=cleaner(sent)[1]
    encoded_sent=[]
    for wd in sent_cl:
        try: 
            encoded_sent.append(Word2Index[wd])
        except: 
            continue   
    input_ids_test.append(encoded_sent)
    
input_ids_val = pad_sequences(input_ids_val,maxlen=MAX_LEN,dtype="long",value=0, truncating="post", padding="post")
input_ids_test = pad_sequences(input_ids_test,maxlen=MAX_LEN,dtype="long",value=0, truncating="post", padding="post")


shape_val=np.asarray(input_ids_val).shape
x_val=np.zeros((shape_val[0], shape_val[1], 200))
i=0
for in_id in input_ids_val:
    x_val[i]=M_GLOVE_space[in_id]
    i+=1
    
shape_test=np.asarray(input_ids_test).shape
x_test=np.zeros((shape_test[0], shape_test[1], 200))
i=0
for in_id in input_ids_test:
    x_test[i]=M_GLOVE_space[in_id]
    i+=1

x_test.shape


(1021, 37, 200)

In [8]:
def predecir_modelos(bs,predichos_all):  #best fit
    temp=predichos_all.copy()
    final=[np.argmax(pred) for pred in temp]  
    confianzas=[temp[i][final[i]] for i in range(len(final))]
    predichos_new=[]
    for i in range(len(temp)):
        indice=final[i]%4
        predichos_new.append(int(indice))
    return predichos_new,final,confianzas    

def predecir_modelos_norm(bs,predichos_all):  #normalizado
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:   
        aux=np.ones(4)
        for i in range(1,6): #for machine pred
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux*np.asarray(sub)
            
        aux=aux/np.sum(aux)
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas
    
def predecir_modelos_average(bs,predichos_all): #average
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:
        aux=np.zeros(4)
        for i in range(1,6):
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux+np.asarray(sub)
            
        aux=aux/5.0 #dividido en el total de machines
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas


In [9]:
del embeddings_index

### 0.0 sin pesos

In [10]:
prob=0.0
path='../new_glove_augmented/stance_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.6868327402135231
F1-macro: 0.5842391323878955
F1-score SDQC: 0.5842391323878955

Committee Norm
Accuracy: 0.7046263345195729
F1-macro: 0.5722039515517776
F1-score SDQC: 0.5722039515517776

Committee Voting
Accuracy: 0.7153024911032029
F1-macro: 0.6020568031173422
F1-score SDQC: 0.6020568031173422
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.66307541625857
F1-macro: 0.4565804672571612
F1-score SDQC: 0.4565804672571612

Committee Norm
Accuracy: 0.7561214495592556
F1-macro: 0.46105320531535177
F1-score SDQC: 0.46105320531535177

Committee Voting
Accuracy: 0.7502448579823702
F1-macro: 0.4

In [11]:
norm_test['none']

[array([0.24285714, 0.15909091, 0.58883249, 0.85343228]),
 array([0.02380952, 0.10126582, 0.41772152, 0.86228937]),
 array([0.12612613, 0.24761905, 0.43312102, 0.85320551]),
 array([0.22033898, 0.17391304, 0.50574713, 0.85404101]),
 array([0.21052632, 0.09876543, 0.43312102, 0.84859366])]

In [12]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.6989226248775711
[TEST] F1-score SDQC Best comité: [0.18813701 0.20515302 0.46382251 0.81582908]
[TEST] F1 macro Best comité: 0.41823540456935226
-----------
[TEST] Acc Norm comité: 0.7545543584720862
[TEST] F1-score SDQC Norm comité: [0.16473162 0.15613085 0.47570863 0.85431237]
[TEST] F1 macro Norm comité: 0.41272086741755826
-----------
[TEST] Acc Voting comité: 0.7559255631733595
[TEST] F1-score SDQC Voting comité: [0.1224249  0.16118902 0.49242577 0.85571564]
[TEST] F1 macro Voting comité: 0.4079388346767108


### 0.0 cw

In [13]:
prob=0.0
path='../new_glove_augmented/stance_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'_cw/cnn1_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'_cw/cnn2_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'_cw/rnn1_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'_cw/rnn2_'+str(j)+'-exec.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'_cw/rnn3_'+str(j)+'-exec.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.604982206405694
F1-macro: 0.5101315017126793
F1-score SDQC: 0.5101315017126793

Committee Norm
Accuracy: 0.6797153024911032
F1-macro: 0.5667279611882622
F1-score SDQC: 0.5667279611882622

Committee Voting
Accuracy: 0.6548042704626335
F1-macro: 0.540050066768629
F1-score SDQC: 0.540050066768629
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.6072477962781586
F1-macro: 0.4080003675688219
F1-score SDQC: 0.4080003675688219

Committee Norm
Accuracy: 0.7159647404505387
F1-macro: 0.4583285235820511
F1-score SDQC: 0.4583285235820511

Committee Voting
Accuracy: 0.693437806072478
F1-macro: 0.4594721958182371
F1-score SDQC: 0.4594721958182371

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accu

In [14]:
norm_test['none']

[array([0.13559322, 0.28742515, 0.58653846, 0.82375726]),
 array([0.17218543, 0.24460432, 0.58      , 0.81829897]),
 array([0.18518519, 0.28125   , 0.58585859, 0.85323383]),
 array([0.15267176, 0.22018349, 0.54651163, 0.84539877]),
 array([0.17322835, 0.33898305, 0.57627119, 0.81091877])]

In [15]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.6579823702252694
[TEST] F1-score SDQC Best comité: [0.14356289 0.2336784  0.51980306 0.7807624 ]
[TEST] F1 macro Best comité: 0.41945168830214663
-----------
[TEST] Acc Norm comité: 0.725171400587659
[TEST] F1-score SDQC Norm comité: [0.16377279 0.2744892  0.57503597 0.83032152]
[TEST] F1 macro Norm comité: 0.4609048706944312
-----------
[TEST] Acc Voting comité: 0.7122428991185114
[TEST] F1-score SDQC Voting comité: [0.17422817 0.27404515 0.61543953 0.81924028]
[TEST] F1 macro Voting comité: 0.47073828163553355


### 0.15

In [16]:
prob=0.15
path='../new_glove_augmented/stance_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec_app1_Top_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.608540925266904
F1-macro: 0.49764221479337756
F1-score SDQC: 0.49764221479337756

Committee Norm
Accuracy: 0.6654804270462633
F1-macro: 0.5370115995115995
F1-score SDQC: 0.5370115995115995

Committee Voting
Accuracy: 0.6548042704626335
F1-macro: 0.5323753530494866
F1-score SDQC: 0.5323753530494866
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.6679725759059746
F1-macro: 0.39625098831448763
F1-score SDQC: 0.39625098831448763

Committee Norm
Accuracy: 0.7375122428991185
F1-macro: 0.4188669091935424
F1-score SDQC: 0.4188669091935424

Committee Voting
Accuracy: 0.7316356513222331
F1-macro: 0.420100160414801
F1-score SDQC: 0.420100160414801

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit

In [17]:
norm_test['none']

[array([0.03846154, 0.17460317, 0.61818182, 0.84422111]),
 array([0.09150327, 0.24793388, 0.53333333, 0.82745592]),
 array([0.13533835, 0.23188406, 0.53631285, 0.83919598]),
 array([0.05769231, 0.14285714, 0.53475936, 0.84441733]),
 array([0.0990099 , 0.15217391, 0.5257732 , 0.84954683])]

In [18]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.6773751224289912
[TEST] F1-score SDQC Best comité: [0.0985273  0.20060576 0.49484766 0.80304865]
[TEST] F1 macro Best comité: 0.39925734123493967
-----------
[TEST] Acc Norm comité: 0.7326150832517139
[TEST] F1-score SDQC Norm comité: [0.08440107 0.18989043 0.54967211 0.84096743]
[TEST] F1 macro Norm comité: 0.4162327624426492
-----------
[TEST] Acc Voting comité: 0.7330068560235062
[TEST] F1-score SDQC Voting comité: [0.09291343 0.20018159 0.56438427 0.84030882]
[TEST] F1 macro Voting comité: 0.42444702502263376


### 0.5

In [19]:
prob=0.5
path='../new_glove_augmented/stance_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec_app1_Top_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.5551601423487544
F1-macro: 0.49275237282543705
F1-score SDQC: 0.49275237282543705

Committee Norm
Accuracy: 0.6227758007117438
F1-macro: 0.5302195340501793
F1-score SDQC: 0.5302195340501793

Committee Voting
Accuracy: 0.6512455516014235
F1-macro: 0.5443253199483385
F1-score SDQC: 0.5443253199483385
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.435847208619001
F1-macro: 0.34583873946675536
F1-score SDQC: 0.34583873946675536

Committee Norm
Accuracy: 0.5886385896180215
F1-macro: 0.4071890253255982
F1-score SDQC: 0.4071890253255982

Committee Voting
Accuracy: 0.6307541625857003
F1-macro: 0.417344804389277
F1-score SDQC: 0.417344804389277

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit

In [20]:
norm_test['none']

[array([0.10810811, 0.24460432, 0.5483871 , 0.72765658]),
 array([0.10071942, 0.14814815, 0.47674419, 0.82686383]),
 array([0.08433735, 0.23703704, 0.53763441, 0.81157556]),
 array([0.08484848, 0.17094017, 0.47191011, 0.8091024 ]),
 array([0.14173228, 0.22222222, 0.50549451, 0.81858689])]

In [21]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.6088148873653281
[TEST] F1-score SDQC Best comité: [0.10545953 0.19415911 0.48948341 0.74507613]
[TEST] F1 macro Best comité: 0.38354454436639995
-----------
[TEST] Acc Norm comité: 0.6752203721841332
[TEST] F1-score SDQC Norm comité: [0.10394913 0.20459038 0.50803406 0.79875705]
[TEST] F1 macro Norm comité: 0.4038326559394756
-----------
[TEST] Acc Voting comité: 0.6863858961802154
[TEST] F1-score SDQC Voting comité: [0.09682584 0.21001821 0.52714224 0.8078121 ]
[TEST] F1 macro Voting comité: 0.410449599127396


### 0.85

In [22]:
prob=0.85
path='../new_glove_augmented/stance_baselines/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(1,6):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j)+'-exec_app1_Top_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j)+'-exec_app1_Top_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.6227758007117438
F1-macro: 0.5064119675711178
F1-score SDQC: 0.5064119675711178

Committee Norm
Accuracy: 0.6797153024911032
F1-macro: 0.5390506630155558
F1-score SDQC: 0.5390506630155558

Committee Voting
Accuracy: 0.6725978647686833
F1-macro: 0.5293481122370457
F1-score SDQC: 0.5293481122370457
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.6748285994123409
F1-macro: 0.39340850114673864
F1-score SDQC: 0.39340850114673864

Committee Norm
Accuracy: 0.7081292850146915
F1-macro: 0.3958321036867881
F1-score SDQC: 0.3958321036867881

Committee Voting
Accuracy: 0.7110675808031341
F1-macro: 0.41116646570331356
F1-score SDQC: 0.41116646570331356

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best 

In [23]:
norm_test['none']

[array([0.10884354, 0.16071429, 0.48648649, 0.82728411]),
 array([0.08474576, 0.16216216, 0.53038674, 0.83823529]),
 array([0.06802721, 0.23602484, 0.54368932, 0.79450262]),
 array([0.08474576, 0.17054264, 0.56410256, 0.83      ]),
 array([0.09722222, 0.20472441, 0.5026738 , 0.81944444])]

In [24]:
m=5.0

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/m))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/m)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/m))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/m)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/m))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/m))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/m)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/m))


[TEST] Acc Best comité: 0.6575905974534769
[TEST] F1-score SDQC Best comité: [0.0937964  0.18084757 0.49464667 0.78703429]
[TEST] F1 macro Best comité: 0.3890812326513179
-----------
[TEST] Acc Norm comité: 0.7065621939275221
[TEST] F1-score SDQC Norm comité: [0.0887169  0.18683367 0.52546778 0.82189329]
[TEST] F1 macro Norm comité: 0.40572791016224186
-----------
[TEST] Acc Voting comité: 0.706366307541626
[TEST] F1-score SDQC Voting comité: [0.09054231 0.19308136 0.53721733 0.82107484]
[TEST] F1 macro Voting comité: 0.4104789599427998
